## Datenaufbereitung

### Überblick

In diesem Notebook werden die Daten aufbereitet. Die Auswahl der Methoden wurden in Kapitel 10 begründet.
- Fehlende Werte (missing values)
- Categorical Encoding
- Ausreißer Handling
- Feature Scaling
- Feature Creation

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import csv
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import category_encoders as ce
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

### Load Data

In [3]:
df = pd.read_hdf('Data/data.h5',key='df')

In [4]:
# separate scale levels
metric_features = ['QUOTE_PRICE_ASK', 'SPREAD', 'COUPON', 'MATURITY_REMAINING', 'QUANTITY1', 'MID', 'PRICE','INTERBANK_POINTS','SALES_FEE']

ordinal_features=['BOND_RATING_MOODYS','COMPETITION_COUNT']

categorical_features =  ['CUSTOMER_USER', 'CUSTOMER_USER_ID', 'CUSTOMER_COMPANY', 'CUSTOMER_COMPANY_ID', 'CUSTOMER_GROUP','CUSTOMER_CLASSIFICATION',
                       'INSTRUMENT_IDENTIFIER', 'INSTRUMENT_GROUP_ID', 'INSTRUMENT_BOOK', 'MARKET_NAME','NOTIONAL_CURRENCY', 'SALES_GROUP', 'SALES_USER', 
                       'SECTOR','CUSTOMER_SUBACCOUNT_ID', 'CUSTOMER_COUNTRY', 'POPUP_REASON_EXISTS']

binary_features = ['POPUP_QUOTE_REFLEX_STATUS', 'POPUP_QUOTE_SIZE_EXCEEDED','POPUP_LIMIT_EXCEEDED', 'AXE_WAS_ACTIVE', 'LIQUID', 'SI','IS_CONSULTANCY']

y='TRADE'

### Missing Values

In [27]:
(df.isna().sum()/df.shape[0]).sort_values(ascending=False)

MID                          0.723453
COUPON                       0.644904
SI                           0.636597
LIQUID                       0.636597
AXE_WAS_ACTIVE               0.462204
CUSTOMER_GROUP               0.396370
SPREAD                       0.195852
PRICE                        0.124100
CUSTOMER_CLASSIFICATION      0.100430
QUANTITY1                    0.094077
QUOTE_PRICE_ASK              0.087745
BOND_RATING_MOODYS           0.087031
MARKET_NAME                  0.080520
INSTRUMENT_BOOK              0.031844
SECTOR                       0.011695
POPUP_LIMIT_EXCEEDED         0.008491
POPUP_QUOTE_REFLEX_STATUS    0.008491
POPUP_REASON_EXISTS          0.008491
POPUP_QUOTE_SIZE_EXCEEDED    0.008491
MATURITY_REMAINING           0.008466
NOTIONAL_CURRENCY            0.004778
INSTRUMENT_GROUP_ID          0.004026
SALES_GROUP                  0.002690
CUSTOMER_USER_ID             0.000564
CUSTOMER_COUNTRY             0.000033
CUSTOMER_COMPANY_ID          0.000004
CUSTOMER_COM

In [28]:
# replace missing values in categorical and ordinal features by -1

df[categorical_features+ordinal_features+binary_features] = df[categorical_features+ordinal_features+binary_features].fillna(-1)

# replace missing values in metric features by mean
for col in metric_features:
    mean = df[col].mean()
    df[col] = df[col].fillna(mean)

### Categorical and Binary Feature Encoding

In [30]:
le = preprocessing.LabelEncoder()
for c in (categorical_features+binary_features):
    le.fit(df[c])
    df[c] = le.transform(df[c])
df[categorical_features]

,CUSTOMER_USER,CUSTOMER_USER_ID,CUSTOMER_COMPANY,CUSTOMER_COMPANY_ID,CUSTOMER_GROUP,CUSTOMER_CLASSIFICATION,INSTRUMENT_IDENTIFIER,INSTRUMENT_GROUP_ID,INSTRUMENT_BOOK,MARKET_NAME,NOTIONAL_CURRENCY,SALES_GROUP,SALES_USER,SECTOR,CUSTOMER_SUBACCOUNT_ID,CUSTOMER_COUNTRY,POPUP_REASON_EXISTS
341398,6925,7296,913,913,14,1,8900,4,15,4,3,58,0,7,1231,7,2
254717,3129,1205,576,576,3,2,8188,10,27,1,3,13,15,64,869,18,2
1154761,8805,558,687,687,3,1,8873,10,35,1,3,29,95,58,980,7,2
212899,6039,4812,927,927,5,0,7767,1,87,1,3,4,68,53,1247,14,2
1314406,5932,5300,634,634,3,8,6378,3,24,1,3,32,81,75,932,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243894,8747,5137,957,957,0,5,8609,14,18,6,3,8,88,59,1276,14,2
1417194,6911,3646,1355,1355,0,6,3594,8,102,0,3,35,45,37,1849,17,1
1356122,10186,6618,609,609,3,0,7375,2,100,1,3,36,47,44,905,19,2
779947,3272,6095,1165,1165,7,3,3638,3,12,5,3,16,41,41,674,13,2


### Ordinal Feature Encoding

In [31]:
df['BOND_RATING_MOODYS'].value_counts()

-1      20837
B2      20823
B3      20662
B1      20648
Aa2     19827
Aa1     19811
Aaa     19663
C1      13418
C       13218
Ca      13004
Caa3     8273
Caa2     8151
Caa1     8149
Ba3      6378
Ba2      6254
Ba1      6106
Baa2     2471
Baa1     2466
Baa3     2433
A3       2334
A2       2262
A1       2232
Name: BOND_RATING_MOODYS, dtype: int64

In [32]:
moodys_map = {
    **dict.fromkeys(['Aaa','Aa1','Aa2','Aa3'], 0), 
    **dict.fromkeys(['A1','A2','A3'], 1),
    **dict.fromkeys(['Baa1','Baa2','Baa3'], 2), 
    **dict.fromkeys(['Ba1','Ba2','Ba3'], 3), 
    **dict.fromkeys(['B1','B2','B3'], 4),
    **dict.fromkeys(['Caa1','Caa2','Caa3'], 5),
    **dict.fromkeys(['Ca','C1','C'], 6),
    -1:-1
}

df['BOND_RATING_MOODYS'] = df['BOND_RATING_MOODYS'].map(moodys_map)

In [33]:
df['BOND_RATING_MOODYS'].value_counts()

 4    62133
 0    59301
 6    39640
 5    24573
-1    20837
 3    18738
 2     7370
 1     6828
Name: BOND_RATING_MOODYS, dtype: int64

In [34]:
df['COMPETITION_COUNT'].value_counts()

5     77507
6     56280
4     20246
9     19828
3     14526
1     10740
2      8195
8      8140
7      7519
15     6022
10     3266
12     2575
11     1820
13     1372
14     1313
0        71
Name: COMPETITION_COUNT, dtype: int64

### Outlier Handling

In [35]:
def removeOutliners(ft,val):
    ft.clip(np.percentile(ft,val), np.percentile(ft,100-val), inplace=True)
    return ft

for col in metric_features:
    df[col] = removeOutliners(df[col],1)

### Metric Feature Scaling

In [36]:
scaler = MinMaxScaler()
scaled_values = scaler.fit_transform(df[metric_features]) 
df[metric_features]=scaled_values

### Export Data

In [ ]:
df.to_hdf('preprocessed_data.h5', key='df', mode='w')
x_train.to_hdf('preprocessed_data.h5', key='x_train')
x_test.to_hdf('preprocessed_data.h5', key='x_test')
y_train.to_hdf('preprocessed_data.h5', key='y_train')
y_test.to_hdf('preprocessed_data.h5', key='y_test')